# Языковые модели

Языковые модели играют важную роль в системах распознавания речи, помогая создавать более грамотные и лексически корректные тексты. В данной работе мы будем изучать нграмные языковые модели, которые позволяют довольно легко оценить вероятность и правдоподобность текста.

В нграмной языковой модели, нграм - это последовательность из n слов в тексте. Например, в предложении "по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас", биграмами будут "по-моему мы", "мы сэкономим", "сэкономим уйму" итд. Языковые модели оценивают вероятность появления последовательности слов, исходя из статистики появления каждого из нграм в обучающей выборке.

Порядком (order) нграм языковой модели называют максимальную длину нграм, которую учитывает модель. 

Практическая работа разделена на 2 части: 
1. Построение нграмой языковой модели - основная часть, 12 баллов
1. Предсказание с помощью языковой модели - дополнительная часть, 4 балла



Полезные сслыки:
* arpa формат - https://cmusphinx.github.io/wiki/arpaformat/
* обучающие материалы - https://pages.ucsd.edu/~rlevy/teaching/2015winter/lign165/lectures/lecture13/lecture13_ngrams_with_SRILM.pdf
* обучающие материалы.2 - https://cjlise.github.io/machine-learning/N-Gram-Language-Model/

In [1]:
import numpy as np
from collections import defaultdict
from typing import List, Dict, Tuple

# 1. Построение нграмной языковой модели. (12 баллов)


Вероятность текста с помощью нграмной языковой модели можно вычислить по формуле: 
$$ P(w_1, w_2, .., w_n) = {\prod{{P_{i=0}^{n}(w_i| w_{i-order}, .., w_{i-1})}}} $$

В простом виде, при обучении нграмной языковой модели, чтобы рассчитать условную вероятность каждой нграмы, используется формула, основанная на количестве появлений нграмы в обучающей выборке. Формула выглядит следующим образом:
$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i})} \over {count(w_{i-order},..., w_{i-1})}} $$

Поскольку униграмы не содержат в себе какого-дибо контекста, вероятность униграмы можно посчитать поделив кол-во этой слова на общее количество слов в обучающей выборке. 


In [2]:
# # в первую очередь нам понадобится подсчитать статистику по обучающей выборке 
# def count_ngrams(train_text: List[str], order=3, bos=True, eos=True) -> Dict[Tuple[str], int]:
#     ngrams = defaultdict(int)
#     # TODO реализуйте функцию, которая подсчитывает все 1-gram 2-gram ... order-gram ngram'ы в тексте
    
#     # 
#     return dict(ngrams)

In [3]:
def count_ngrams(train_text: List[str], order=3, bos=True, eos=True) -> Dict[Tuple[str], int]:
    ngrams = defaultdict(int)

    for text in train_text:
        words = text.split()
        words = bos * ['<s>'] + words + eos * ['</s>']

        for suborder in range(1, order + 1):
            ngrams_count = len(words) - (suborder - 1)
            for j in range(ngrams_count):
                ngrams[tuple(words[j:j + suborder])] += 1

    return dict(ngrams)

In [4]:
def test_count_ngrams():
    assert count_ngrams(['привет привет как дела'], order=1, bos=True, eos=True) == {
        ('<s>',): 1, 
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=True) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=2, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1,
        ('привет', 'привет'): 1,
        ('привет', 'как'): 1,
        ('как', 'дела'): 1
    }    
    assert count_ngrams(['привет ' * 6], order=2, bos=False, eos=False) == {
        ('привет',): 6, 
        ('привет', 'привет'): 5
    }
    result = count_ngrams(['практическое сентября',
                           'второе практическое занятие пройдет в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=5)
    assert result[('<s>',)] == 3
    assert result[('32',)] == 3
    assert result[('<s>', 'в', 'офлайне', 'в', '32')] == 1
    assert result[('офлайне', 'в', '32', '12', '</s>')] == 1
    print('Test 1a passed')
    
    
test_count_ngrams()  

Test 1a passed



Простой подход к вычислению вероятностей через количество нграм имеет существенный недостаток. Если в тексте встретится нграмма, которой не было в обучающей выборке, то вероятность всего текста будет равна нулю. 

Чтобы избежать данного недостатка, вводится специальное сглаживание - [сглаживание Лапласа](https://en.wikipedia.org/wiki/Additive_smoothing). Данная техника позволяет учитывать нграмы, не встретившиеся в обучающей выборке, и при этом не делает вероятность текста равной нулю.

Формула сглаживания Лапласа выглядит следующим образом:

$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i}) + k} \over {count(w_{i-order},..., w_{i-1}) + k*V}} $$

Здесь V - количество слов в словаре, а k - гиперпараметр, который контролирует меру сглаживания. Как правило, значение k выбирается экспериментально, чтобы найти оптимальный баланс между учетом редких нграм и сохранением вероятности для часто встречающихся нграм.


In [5]:
# # функция подсчета вероятности через количество со сглаживанием Лапласа
# def calculate_ngram_prob(ngram: Tuple[str], counts: Dict[Tuple[str], int], V=None, k=0) -> float:
#     # подсчитывет ngram со сглаживанием Лапласа
#     # TODO
#     return prob

In [6]:
def calculate_ngram_prob(ngram: Tuple[str], counts: Dict[Tuple[str], int], V=None, k=0) -> float:
    if V is None:
        V = len([ngram for ngram in counts if len(ngram) == 1])

    ngram_count = counts.get(ngram, 0)
    sub_ngrams_count = sum([counts[ngram] for ngram in counts if len(ngram) == 1])
    if len(ngram) > 1:
        sub_ngrams_count = counts[ngram[:-1]] if ngram[:-1] in counts else 0 #  + (V * k)

    # print(f"({ngram_count} + {k}) / ({sub_ngrams_count} + {k} * {V})")
    prob = (ngram_count + k) / (sub_ngrams_count + k * V)
    return prob

In [7]:
counts = count_ngrams(['практическое сентября', 'второе практическое занятие в офлайне 32 сентября в 12 часов 32 минуты', 'в офлайне в 32 12'], order=4)
# {ngram: counts[ngram] for ngram in counts if len(ngram) == 2}

In [8]:
calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25

True

In [9]:
def test_calculate_ngram_prob():
    counts = count_ngrams(['практическое сентября',
                           'второе практическое занятие в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=4)
    assert calculate_ngram_prob(('в', 'офлайне'), counts) == 0.5
    assert calculate_ngram_prob(('в', ), counts) == 4/25
    assert calculate_ngram_prob(('в', ), counts, k=0.5) == (4+0.5)/(25+0.5*12)
    assert calculate_ngram_prob(('в', 'офлайне', 'в', '32'), counts) == 1.0
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=1) == 0.1875
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=0) == 0.0
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=1) == 0.0625
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25

    print("Test 1.b passed")
    

test_calculate_ngram_prob()  

Test 1.b passed


Основной метрикой язковых моделей является перплексия. 

Перплексия  — безразмерная величина, мера того, насколько хорошо распределение вероятностей предсказывает выборку. Низкий показатель перплексии указывает на то, что распределение вероятности хорошо предсказывает выборку.

$$ ppl = {P(w_1, w_2 ,..., w_N)^{- {1} \over {N}}} $$


In [10]:
# Языковая модель 
class NgramLM:
    def __init__(self, order=3, bos=True, eos=True, k=1, predefined_vocab=None):
        self.order = order
        self.eos = eos
        self.bos = bos
        self.k = k
        self.vocab = predefined_vocab
        self.ngrams_count = None
        
    @property
    def V(self) -> int:
        return len(self.vocab)
    
    def fit(self, train_text: List[str]) -> None:
        self.ngrams_count = count_ngrams(train_text, self.order, self.bos, self.eos)
        self.vocab = [ngram[0] for ngram in self.ngrams_count if len(ngram) == 1]
    
    def predict_ngram_log_proba(self, ngram: Tuple[str]) -> float:
        proba = calculate_ngram_prob(ngram, self.ngrams_count, V=self.V, k=self.k)
        log_proba = np.log(proba)
        return log_proba
           
    def predict_log_proba(self, words: List[str]) -> float:
        words = self.bos * ['<s>'] + words[0].split() + self.eos * ['</s>']
        logprob = 0
        start, end = 1 - self.order, 0
        while end != len(words):
            ngram = tuple(words[start if start > 0 else 0 : end + 1])
            logprob += self.predict_ngram_log_proba(ngram)
            start, end = start + 1, end + 1
        return logprob
        
    def ppl(self, text: List[str]) -> float:
        logprob = self.predict_log_proba(text)
        ngrams_count = sum([len(line.split()) + self.bos + self.eos for line in text])
        perplexity = np.exp(-logprob / ngrams_count)
        return perplexity

In [11]:
train_data = ["по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас",
                  "если я сойду с ума прямо сейчас по-моему мы сэкономим уйму времени",
                  "мы сэкономим уйму времени если я сейчас сойду с ума по-моему"]
lm = NgramLM(order=2)
lm.fit(train_data)

In [12]:
print(lm.ppl(['']), ((3+1)/(41 + 14) * 1/(3+14))**(-1/2))
print(lm.ppl(['ЧТО']), ((3+1)/(41 + 14) * 1/(3+14) * 1/(14)) ** (-1/3))
print(lm.ppl(["по-моему если я прямо сейчас сойду с ума мы сэкономим уйму времени"]), 7.33)

15.288884851420658 15.288884851420656
14.846584407951667 14.84658440795166
7.334561964590593 7.33


In [13]:
def test_lm():
    train_data = ["по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас",
                  "если я сойду с ума прямо сейчас по-моему мы сэкономим уйму времени",
                  "мы сэкономим уйму времени если я сейчас сойду с ума по-моему"]
    global lm
    lm = NgramLM(order=2)
    lm.fit(train_data)
    assert lm.V == 14
    assert np.isclose(lm.predict_log_proba(['мы']), lm.predict_log_proba(["если"]))
    assert lm.predict_log_proba(["по-моему"]) > lm.predict_log_proba(["если"]) 
    
    gt = ((3+1)/(41 + 14) * 1/(3+14))**(-1/2)
    ppl = lm.ppl([''])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    gt = ((3+1)/(41 + 14) * 1/(3+14) * 1/(14)) ** (-1/3)
    ppl = lm.ppl(['ЧТО'])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    test_data = ["по-моему если я прямо сейчас сойду с ума мы сэкономим уйму времени"]
    ppl = lm.ppl(test_data)
    assert round(ppl, 2) == 7.33, f"{ppl}"
test_lm()

# 2. Предсказания с помощью языковой модели (4 балла)

In [14]:
# def predict_next_word(lm: NgramLM, prefix: List[str], topk=4):
#     # TODO реализуйте функцию, которая предсказывает продолжение фразы. 
#     # верните topk наиболее вероятных продолжений фразы prefix 
#     return word

In [15]:
def predict_next_word(lm: NgramLM, prefix: List[str], topk=4):
    probabilities = list()
    for word in lm.vocab:
        ngram = tuple(prefix + [word])
        proba = lm.predict_ngram_log_proba(ngram)
        probabilities.append((proba, word))
    topk_words = [item[1] for item in sorted(probabilities, reverse=True)[:topk]]
    return topk_words

Обучите языковую модель на всем тексте из этой лабораторной работы (не забудьте заранее нормализовать текст).

Что предскажет модель по префиксам `по-моему`, `сэкономим`, `нграм` и `вероятности`? 

In [16]:
train_text = """
Языковые модели играют важную роль в системах распознавания речи, помогая создавать более грамотные и лексически корректные тексты.
В данной работе мы будем изучать нграмные языковые модели, которые позволяют довольно легко оценить вероятность и правдоподобность текста.
В нграмной языковой модели, нграм - это последовательность из n слов в тексте. 
Например, в предложении "по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас", биграмами будут "по-моему мы", "мы сэкономим", "сэкономим уйму" и тд.
Языковые модели оценивают вероятность появления последовательности слов, исходя из статистики появления каждого из нграм в обучающей выборке.
Порядком (order) нграм языковой модели называют максимальную длину нграм, которую учитывает модель.
Практическая работа разделена на 2 части:
Построение нграмой языковой модели - основная часть, 12 баллов.
Предсказание с помощью языковой модели - дополнительная часть, 4 балла.
Полезные сслыки.
Вероятность текста с помощью нграмной языковой модели можно вычислить по формуле.
В простом виде, при обучении нграмной языковой модели, чтобы рассчитать условную вероятность каждой нграмы, используется формула, основанная на количестве появлений нграмы в обучающей выборке. 
Формула выглядит следующим образом.
Простой подход к вычислению вероятностей через количество нграм имеет существенный недостаток. 
Если в тексте встретится нграмма, которой не было в обучающей выборке, то вероятность всего текста будет равна нулю. 
Чтобы избежать данного недостатка, вводится специальное сглаживание - сглаживание Лапласа.
Данная техника позволяет учитывать нграмы, не встретившиеся в обучающей выборке, и при этом не делает вероятность текста равной нулю.
Формула сглаживания Лапласа выглядит следующим образом.
Здесь V - количество слов в словаре, а k - гиперпараметр, который контролирует меру сглаживания.
Как правило, значение k выбирается экспериментально, чтобы найти оптимальный баланс между учетом редких нграм и сохранением вероятности для часто встречающихся нграм.
Основной метрикой язковых моделей является перплексия.
Перплексия  — безразмерная величина, мера того, насколько хорошо распределение вероятностей предсказывает выборку.
Низкий показатель перплексии указывает на то, что распределение вероятности хорошо предсказывает выборку.
Обучите языковую модель на всем тексте из этой лабораторной работы (не забудьте заранее нормализовать текст).
Что предскажет модель по префиксам `по-моему`, `сэкономим`, `нграм` и `вероятности`? 
"""

In [17]:
import re

def process_text(text: str):
    sentences = text.strip().split("\n")
    processed_sentences = list()
    for sentence in sentences:
        sentence = sentence.strip().lower()
        sentence = re.sub(r"[^\w\d\-\s]", " ", sentence).strip()
        sentence = re.sub(r"\s+-\s+", " ", sentence)
        sentence = re.sub(r"\s+", " ", sentence)
        processed_sentences.append(sentence)
    return processed_sentences

In [18]:
train_sentences = process_text(train_text)
lm = NgramLM(order=2)
lm.fit(train_sentences)

In [19]:
print("по-моему", predict_next_word(lm, ["по-моему"], topk=4))
print("сэкономим", predict_next_word(lm, ["сэкономим"], topk=4))
print("нграм", predict_next_word(lm, ["нграм"], topk=4))
print("вероятности", predict_next_word(lm, ["вероятности"], topk=4))

по-моему ['мы', 'сэкономим', 'языковые', 'языковую']
сэкономим ['уйму', 'сэкономим', 'нграм', 'языковые']
нграм ['и', 'языковой', 'это', 'которую']
вероятности ['хорошо', 'для', '</s>', 'языковые']
